# __Set Up & Import__

## Import Necessary Library

In [ ]:
# %pip install opencv-python
# %pip install mediapipe
# %pip install plotly
# %pip install pandas

In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import os
import time 


/Users/theerat/Documents/tsl/sign-language-translator/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Set Color Class [ _Not Important_ ]

In [2]:
class color:
   PINK = '\033[95m'
   CYAN = '\033[96m'
   PURPLE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

## Set Up Detection Model

In [3]:
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
    min_detection_confidence=0.1,
    min_tracking_confidence=0.1
)

# Initializing the drawing utils for drawing the landmarks on image
mp_drawing = mp.solutions.drawing_utils

I0000 00:00:1732439418.200276  130942 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1732439418.329169  134157 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732439418.344696  134157 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732439418.348378  134153 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732439418.348968  134156 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732439418.349114  134159 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732439418.361790  134

# __Define Needed Function__

## Extract & Structerize Position From Each Part

In [30]:
def extract_data(hand_pos):
    right, left, body, face, i = dict(), dict(), dict(), dict(),0
    if hand_pos[0] != None:
        for data in hand_pos[0].landmark:
            right['Landmark '+str(i)] = [data.x, data.y, data.z]
            i+=1
        i = 0
    else:
        right = None

    if hand_pos[1] != None:
        for data in hand_pos[1].landmark:
            left['Landmark '+str(i)] = [data.x, data.y, data.z]
            i+=1
        i = 0
    else:
        left = None

    if hand_pos[2] != None:
        for data in hand_pos[2].landmark:
            body['Landmark '+str(i)] = [data.x, data.y, data.z]
            i+=1
        i = 0
    else:
        body = None

    if hand_pos[3] != None:
        for data in hand_pos[3].landmark:
            face['Landmark '+str(i)] = [data.x, data.y, data.z]
            i+=1
        i = 0
    else:
        face = None

    hand_pos = {'Right': right, 'Left': left, 'Body': body, 'Face': face}
    hand_pos_df = pd.DataFrame(hand_pos).T
    
    return hand_pos_df

## Main Function

In [5]:
def operate(mp_holistic,holistic_model, mp_drawing, file):    
    vid = cv2.VideoCapture(file)
    total_frame, total_capture = int(vid.get(cv2.CAP_PROP_FRAME_COUNT)), 0

    for i in range(total_frame):
        if i%(total_frame//30) == 0:
            total_capture+=1
        else:
            continue
        ret, frame = vid.read()

        vid.set(cv2.CAP_PROP_POS_FRAMES, (total_capture)*(total_frame/30))

        if not ret or cv2.waitKey(5) & 0xFF == ord('q'):
            break
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        image.flags.writeable = False
        results = holistic_model.process(image)
        image.flags.writeable = True
    
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw Pose And Face Land Marks 
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        # Drawing Right hand Land Marks
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        # Drawing Left hand Land Marks
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        # Draw Face Land Marks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)

        # Display the resulting image
        cv2.imshow("Display", image)

        left= results.left_hand_landmarks
        right = results.right_hand_landmarks
        upper = results.pose_landmarks
        face = results.face_landmarks
        hand_pos_df = extract_data([right, left, upper, face])
        os.makedirs(file.split('.')[0], exist_ok=True)
        hand_pos_df.to_csv(file.split('.')[0]+'/Frame_'+str(total_capture-1)+'.csv')
        print("\r"+"Successfully captured frame:", color.RED+color.BOLD+str(total_capture)+color.END+" / 30", end="")
        # print("Successfully captured frame:", color.RED+color.BOLD+str(total_capture)+color.END+" / 30")

    vid.release()
    cv2.destroyAllWindows()
    print("\n"+color.BOLD+color.GREEN+"Done!"+color.END)

In [6]:
def extract_data2(hand_pos):
    data = {"Right": [], "Left": [], "Body": [], "Face": []}
    keys = ["Right", "Left", "Body", "Face"]

    for idx, landmarks in enumerate(hand_pos):
        if landmarks:
            data[keys[idx]] = [[lm.x, lm.y, lm.z] for lm in landmarks.landmark]
        else:
            data[keys[idx]] = None

    return pd.DataFrame(data)

In [7]:
def operate_test(mp_holistic, holistic_model, mp_drawing, file):    
    vid = cv2.VideoCapture(file)
    total_frame = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    total_capture = 0

    os.makedirs(file.split('.')[0], exist_ok=True)  # สร้างโฟลเดอร์สำหรับบันทึกผลลัพธ์

    for i in range(30):  # จับภาพ 30 เฟรม
        frame_position = int((i / 30) * total_frame)
        vid.set(cv2.CAP_PROP_POS_FRAMES, frame_position)
        ret, frame = vid.read()

        if not ret:
            break
        
        # แปลงภาพเป็น RGB เพื่อให้ใช้กับ MediaPipe ได้
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = holistic_model.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # วาดจุด landmark บนภาพ
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)

        # แสดงภาพที่มี landmark
        cv2.imshow("Display", image)

        # ดึงข้อมูลจากจุด landmark
        hand_pos_df = extract_data([
            results.right_hand_landmarks,
            results.left_hand_landmarks,
            results.pose_landmarks,
            results.face_landmarks
        ])

        # บันทึกข้อมูลเป็น CSV
        hand_pos_df.to_csv(f"{file.split('.')[0]}/Frame_{i}.csv")
        print(f"\rSuccessfully captured frame: {i+1}/30", end="")

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

    vid.release()
    cv2.destroyAllWindows()
    print("\nDone!")

In [14]:
operate_test(mp_holistic,holistic_model, mp_drawing, '/Users/theerat/Documents/tsl/sign-language-translator/New25/ควาย/video_2.mp4')

W0000 00:00:1732351650.361916  232932 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Successfully captured frame: 30/30
Done!


## Graph Drawing

In [8]:
edges = {
        "left_hand": [
            (0, 1), (1, 2), (2, 3), (3, 4), (0, 5), (0, 17), (5, 6), (6, 7),
            (7, 8), (5, 9), (9, 10), (10, 11), (11, 12), (9, 13), (13, 14),
            (14, 15), (15, 16), (13, 17), (17, 18), (18, 19), (19, 20)
        ],
        "right_hand": [
            (0, 1), (1, 2), (2, 3), (3, 4), (0, 5), (0, 17), (5, 6), (6, 7),
            (7, 8), (5, 9), (9, 10), (10, 11), (11, 12), (9, 13), (13, 14),
            (14, 15), (15, 16), (13, 17), (17, 18), (18, 19), (19, 20)
        ],
        "pose": [
            (8, 6), (6, 5), (6, 4), (4, 0), (0, 1), (1, 2), (2, 3), (3, 7),
            (10, 9), (11, 12), (11, 13), (11, 23), (13, 15), (15, 21), (15, 17),
            (15, 19), (17, 19), (12, 14), (12, 24), (14, 16), (16, 22), (16, 20),
            (16, 18), (18, 20), (23, 24)
        ],
    }

In [9]:
def draw_edges(ax, points, edges):
        for edge in edges:
            start, end = edge
            x_values = [points[start][0], points[end][0]]
            y_values = [points[start][1], points[end][1]]
            ax.plot(x_values, y_values, 'k-')

In [10]:
def graph_drawer(word, frames ,edges):
    frame = pd.read_csv(word+'/frame_'+str(frames)+'.csv')
        
    # Extract data from the CSV file
    lst_frame_right = frame.iloc[0][1:].to_frame().dropna().values
    lst_frame_left = frame.iloc[1][1:].to_frame().dropna().values
    lst_frame_body = frame.iloc[2][1:].to_frame().dropna().values
    lst_frame_face = frame.iloc[3][1:].to_frame().dropna().values

    # Convert string representations of lists to actual lists
    lst_frame_right = [ast.literal_eval(lst_frame_right[i][0]) for i in range(len(lst_frame_right))]
    lst_frame_left = [ast.literal_eval(lst_frame_left[i][0]) for i in range(len(lst_frame_left))]
    lst_frame_body = [ast.literal_eval(lst_frame_body[i][0]) for i in range(len(lst_frame_body))]
    lst_frame_face = [ast.literal_eval(lst_frame_face[i][0]) for i in range(len(lst_frame_face))]

    # Extract x and y coordinates
    right_x = [lst_frame_right[i][0] for i in range(len(lst_frame_right))]
    right_y = [lst_frame_right[i][1] for i in range(len(lst_frame_right))]
    left_x = [lst_frame_left[i][0] for i in range(len(lst_frame_left))]
    left_y = [lst_frame_left[i][1] for i in range(len(lst_frame_left))]
    body_x = [lst_frame_body[i][0] for i in range(len(lst_frame_body))]
    body_y = [lst_frame_body[i][1] for i in range(len(lst_frame_body))]
    face_x = [lst_frame_face[i][0] for i in range(len(lst_frame_face))]
    face_y = [lst_frame_face[i][1] for i in range(len(lst_frame_face))]

    # Create the plot
    _, axis = plt.subplots(2, 2, constrained_layout=True, figsize=(8, 8))
    if right_x != []:
        axis[0, 0].scatter(right_x, right_y)
        axis[0, 0].set_title("Right hand")
        draw_edges(axis[0, 0], lst_frame_right, edges["right_hand"])

    if left_x != []:
        axis[0, 1].scatter(left_x, left_y)
        axis[0, 1].set_title("Left hand")
        draw_edges(axis[0, 1], lst_frame_left, edges["left_hand"])

    if body_x != []:
        axis[1, 0].scatter(body_x, body_y)
        axis[1, 0].set_title("Body")
        draw_edges(axis[1, 0], lst_frame_body, edges["pose"])

    if right_x != [] and left_x != [] and body_x != []:
        axis[1, 1].scatter(right_x, right_y)
        axis[1, 1].scatter(left_x, left_y)
        axis[1, 1].scatter(body_x, body_y)
        axis[1, 1].set_title("All")
        draw_edges(axis[1, 1], lst_frame_right, edges["right_hand"])
        draw_edges(axis[1, 1], lst_frame_left, edges["left_hand"])
        draw_edges(axis[1, 1], lst_frame_body, edges["pose"])

    axis[0,0].yaxis.set_inverted(True)
    axis[0,1].yaxis.set_inverted(True)
    axis[1,0].yaxis.set_inverted(True)
    axis[1,1].yaxis.set_inverted(True)
    plt.show()

# __Using Zone__

In [9]:
operate(mp_holistic,holistic_model, mp_drawing, '/Users/theerat/Documents/tsl/sign-language-translator/New25/กิน/video_6.mp4')

W0000 00:00:1732350140.612258  227507 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Successfully captured frame: 6 / 30

2024-11-23 15:22:21.606 Python[7732:226853] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-23 15:22:21.606 Python[7732:226853] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


Successfully captured frame: 30 / 30
Done!


In [11]:
graph_drawer('hello1 test', 10, edges)

IndexError: single positional indexer is out-of-bounds

# __KeyPoint Folder__

In [51]:
def operate2(mp_holistic, holistic_model, mp_drawing, file, folder_path, index):
    if file == '.DS_Store':
        return

    vdo_path = os.path.join('/Users/theerat/Documents/tsl/sign-language-translator/fix_word', folder_path, file)
    print(f'กำลัง keypoint {vdo_path}...')
    output_dir = os.path.join('Landmarked', folder_path, str(index))
    os.makedirs(output_dir, exist_ok=True)

    vid = cv2.VideoCapture(vdo_path)
    total_frame = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))

    # Avoid ZeroDivisionError if total_frame is less than 30
    capture_interval = total_frame // 30 if total_frame >= 30 else 1

    total_capture = 0

    for i in range(total_frame):
        if i % capture_interval == 0:
            total_capture += 1
        else:
            continue

        ret, frame = vid.read()
        if not ret:
            break

        vid.set(cv2.CAP_PROP_POS_FRAMES, total_capture * (total_frame / 30))
        
        # Convert the frame to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Process the frame with Mediapipe
        results = holistic_model.process(image)
        image.flags.writeable = True
        
        # Convert back to BGR for display
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Draw landmarks on the frame
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
        
        # Display the processed frame (optional)
        cv2.imshow("Display", image)
        
        # Extract and save landmark data
        left = results.left_hand_landmarks
        right = results.right_hand_landmarks
        upper = results.pose_landmarks
        face = results.face_landmarks
        hand_pos_df = extract_data([right, left, upper, face])
        hand_pos_df.to_csv(os.path.join(output_dir, f'{total_capture - 1}.csv'), index=False)
        
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

    # Release resources
    # vid.release()
    # cv2.destroyAllWindows()
    print(f"\nSuccessfully processed file: {file}")


In [52]:
def landmark_folder(folder_path):
    for word_folder in os.listdir(folder_path):
        if word_folder == '.DS_Store':
            continue
        print(f'กำลัง keypoint {word_folder}...')
        files = os.listdir(f'/Users/theerat/Documents/tsl/sign-language-translator/fix_word/{word_folder}')
        os.mkdir('Landmarked/' + word_folder)
        for index,file in enumerate(files):
            operate2(mp_holistic,holistic_model, mp_drawing, file,word_folder,index)
    print("เสร็จหมดละจ้าา")

In [53]:
def landmark_folder(folder_path):
    os.makedirs('Landmarked', exist_ok=True)
    # word_path = ['ไก่ผัดกะเพรา', 'ชอบ', 'หัวเราะ', 'ไม่มี', 'นอน', 'ฉัน', 'นักเรียน', 'วิ่ง']
    word_path = os.listdir(folder_path)
    for word_folder in word_path:
        if word_folder == '.DS_Store':
            continue
        
        print(f'Processing keypoints for {word_folder}...')
        word_folder_path = os.path.join(folder_path, word_folder)
        files = os.listdir(word_folder_path)
        
        os.makedirs(os.path.join('Landmarked', word_folder), exist_ok=True)
        for index, file in enumerate(files):
            operate2(mp_holistic,holistic_model, mp_drawing, file,word_folder,index)
    
    print("All processing complete!")

In [ ]:
word_path = ['ไก่ผัดกะเพรา', 'ชอบ', 'หัวเราะ', 'ไม่มี', 'นอน', 'ฉัน', 'นักเรียน', 'วิ่ง']

['ไก่ผัดกะเพรา', 'ชอบ', 'หัวเราะ', 'ไม่มี', 'นอน', 'ฉัน', 'นักเรียน', 'วิ่ง']

In [54]:
"""
ให้ใส่ชื่อ Folder ที่ต้องการ landmark โดยมันจะไปอยู่ในโฟลเดอร์ Landmarked อีกที ให้สร้างด้วยนะจ๊ะ
พอได้ไฟล์แล้ว เอาไปใส่ในไดร์ฟ อย่าอัปขึน git ไม่งั้น บู้มมมมมมมมมมม 
"""
landmark_folder('/Users/theerat/Documents/tsl/sign-language-translator/fix_word')

Processing keypoints for ฉัน...
กำลัง keypoint /Users/theerat/Documents/tsl/sign-language-translator/fix_word/ฉัน/video_38.mp4...

Successfully processed file: video_38.mp4
กำลัง keypoint /Users/theerat/Documents/tsl/sign-language-translator/fix_word/ฉัน/video_10.mp4...

Successfully processed file: video_10.mp4
กำลัง keypoint /Users/theerat/Documents/tsl/sign-language-translator/fix_word/ฉัน/video_11.mp4...

Successfully processed file: video_11.mp4
กำลัง keypoint /Users/theerat/Documents/tsl/sign-language-translator/fix_word/ฉัน/video_39.mp4...

Successfully processed file: video_39.mp4
กำลัง keypoint /Users/theerat/Documents/tsl/sign-language-translator/fix_word/ฉัน/video_13.mp4...

Successfully processed file: video_13.mp4
กำลัง keypoint /Users/theerat/Documents/tsl/sign-language-translator/fix_word/ฉัน/video_12.mp4...

Successfully processed file: video_12.mp4
กำลัง keypoint /Users/theerat/Documents/tsl/sign-language-translator/fix_word/ฉัน/video_16.mp4...

Successfully processed 

KeyboardInterrupt: 

# __Capture__

In [ ]:
stop_recording = False

def capture(word,count):
    global stop_recording
    webcam = cv2.VideoCapture(0)

    if not webcam.isOpened():
        # print("ไม่สามารถเปิด webcam ได้")
        exit()

    frame_width = int(webcam.get(4))
    frame_height = int(webcam.get(4))

    fps = int(webcam.get(cv2.CAP_PROP_FPS))
    print(f"คำว่า : '{word}' ครั้งที่ {count}")
    # print("Webcam FPS:", fps)

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(f"VDO/{word}/{word}_re_{count}.mp4", fourcc, fps, (frame_width, frame_height))

    while not stop_recording:
        ret, frame = webcam.read()

        if ret:
            frame = cv2.flip(frame, 1)
            pil_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            draw = ImageDraw.Draw(pil_img)

            font_path = "SukhumvitSet-Medium.ttf"
            font = ImageFont.truetype(font_path, 120)
            font2 = ImageFont.truetype(font_path, 50)

            draw.text((50, 50), word, font=font, fill=(255, 255, 255))
            draw.text((50,200), f'คลิปที่ : {count}', font=font, fill=(255, 255, 255))
            draw.text((1500,50), 'กด q เมื่อบันทึกเสร็จ', font=font2, fill=(255, 255, 255))
            draw.text((1500,200), 'กด d เพื่อยกเลิก', font=font2, fill=(255, 255, 255))

            frame = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)

            cv2.imshow("Webcam", frame)
            out.write(frame)

        if cv2.waitKey(100) & 0xFF == ord("q"):
            break

    webcam.release()
    out.release()
    cv2.destroyAllWindows()

In [ ]:
def cap_sl(word):
    global stop_recording
    ex_vdo = cv2.VideoCapture(f'VDO/{word}/{word}_test.mp4')
    while True:
        ret, frame = ex_vdo.read()

        if not ret:
            print("ไม่สามารถอ่านไฟล์ได้")
            break

        pil_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(pil_img)

        font_path = "SukhumvitSet-Medium.ttf"
        font = ImageFont.truetype(font_path, 50)

        text_position = (50, 50)
        draw.text(text_position, word, font=font, fill=(255, 255, 255))

        frame = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
        cv2.imshow('Example Video', frame)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    ex_vdo.release()
    cv2.destroyAllWindows()
    
    time.sleep(3)
    for i in range(1,30):
        capture(word,i)
        time.sleep(5)
        if cv2.waitKey(100) & 0xFF == ord('d'):
            stop_recording = True
            break
        
    print('เสร็จแล้ววว')

In [8]:
import cv2
import time
import os

# Create a directory to save videos
output_dir = "/Users/theerat/Documents/tsl/sign-language-translator/New25/อยู่"
os.makedirs(output_dir, exist_ok=True)

# Open the video capture (0 for default webcam, or provide a video file path)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Cannot open video source.")
    exit()

# Video parameters
fps = 20  # Frames per second
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 files

video_count = 0
start_time = time.time()
out = None

print("Press 'q' to stop capturing videos.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Unable to capture video frame.")
        break

    # Get elapsed time
    elapsed_time = time.time() - start_time

    # Start a new video file every 4 seconds
    if elapsed_time >= 4 or out is None:
        if out:  # Release the current video writer
            out.release()
        
        # Create a new video file
        video_count += 1
        video_filename = os.path.join(output_dir, f"video_{video_count}.mp4")
        out = cv2.VideoWriter(video_filename, fourcc, fps, (frame_width, frame_height))
        print(f"Started new video: {video_filename}")
        start_time = time.time()  # Reset the timer

    # Write the current frame to the video file
    out.write(frame)

    # Display the video count and seconds count on the video feed
    video_text = f"Recording Video: {video_count}"
    seconds_text = f"Elapsed Time: {int(elapsed_time)} sec"

    # ฟอนต์และการตั้งค่า
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 2  # เพิ่มขนาดฟอนต์
    font_color = (0, 255, 0)  # สีเขียว
    thickness = 3  # เพิ่มความหนา

    # ตำแหน่งข้อความ
    video_position = (10, 100)  # ตำแหน่งของข้อความ Video count
    seconds_position = (10, 200)  # ตำแหน่งของข้อความ Elapsed time

    # วาดข้อความลงบนเฟรม
    cv2.putText(frame, video_text, video_position, font, font_scale, font_color, thickness)
    cv2.putText(frame, seconds_text, seconds_position, font, font_scale, font_color, thickness)


    # Display the video feed
    cv2.imshow("Video Feed", frame)

    # Check for the 'q' key to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Exiting...")
        break

# Release the video writer and video capture
if out:
    out.release()
cap.release()
cv2.destroyAllWindows()


Press 'q' to stop capturing videos.
Started new video: /Users/theerat/Documents/tsl/sign-language-translator/New25/อยู่/video_1.mp4
Started new video: /Users/theerat/Documents/tsl/sign-language-translator/New25/อยู่/video_2.mp4
Started new video: /Users/theerat/Documents/tsl/sign-language-translator/New25/อยู่/video_3.mp4
Started new video: /Users/theerat/Documents/tsl/sign-language-translator/New25/อยู่/video_4.mp4
Started new video: /Users/theerat/Documents/tsl/sign-language-translator/New25/อยู่/video_5.mp4
Started new video: /Users/theerat/Documents/tsl/sign-language-translator/New25/อยู่/video_6.mp4
Started new video: /Users/theerat/Documents/tsl/sign-language-translator/New25/อยู่/video_7.mp4
Started new video: /Users/theerat/Documents/tsl/sign-language-translator/New25/อยู่/video_8.mp4
Started new video: /Users/theerat/Documents/tsl/sign-language-translator/New25/อยู่/video_9.mp4
Started new video: /Users/theerat/Documents/tsl/sign-language-translator/New25/อยู่/video_10.mp4
Sta